In [1]:
import os
import sys
import pandas as pd
import numpy as np
import tushare as ts
pro = ts.pro_api('your token')  # 替换成自己的token

In [7]:
#查询当前所有正常上市交易的股票列表
data = pro.stock_basic(exchange='', list_status='L')
data.head()

,ts_code,symbol,name,area,industry,market,list_date
0,000001.SZ,000001,平安银行,深圳,银行,主板,19910403
1,000002.SZ,000002,万科A,深圳,全国地产,主板,19910129
2,000004.SZ,000004,国农科技,深圳,互联网,主板,19910114
3,000005.SZ,000005,世纪星源,深圳,环境保护,主板,19901210
4,000006.SZ,000006,深振业A,深圳,区域地产,主板,19920427


In [43]:
# ==获取前100只股票的信息
stock_df_list = []  # 初始化列表
for code in data['ts_code'].values[:100]:
    # =获取股票历史日线数据
    single_stock_daily = pro.daily(ts_code=code) 
    # 将trade_date转化为日期格式，并排序
    single_stock_daily['trade_date'] = pd.to_datetime(single_stock_daily['trade_date'])
    single_stock_daily.sort_values('trade_date', inplace=True)
    # 将trade_date设置为index
    single_stock_daily.set_index('trade_date', inplace=True)
    
    # =获取股票曾用名
    used_name = pro.namechange(ts_code=code,
                               fields='name,start_date,end_date,change_reason')
    # 将start_date和end_date列内容合并
    used_name = used_name.melt(id_vars=['name', 'change_reason'], 
                               value_vars=['start_date','end_date'],
                               var_name='start_end', value_name='change_date')
    # 转化为日期格式并排序
    used_name['change_date'] = pd.to_datetime(used_name['change_date'])
    used_name.sort_values('change_date', inplace=True)
    # 剔除na值
    used_name = used_name.dropna(subset=['change_date'])
    # 设置成index
    used_name.set_index('change_date', inplace=True)
    
    # =合并曾用名信息和日线信息
    df = pd.merge(single_stock_daily, used_name, 
                  left_index=True, right_index=True,
                  how='outer', sort=True)
    df[['name', 'change_reason']] = df[['name', 'change_reason']].fillna(method='ffill')  # 填充股票名称
    df.dropna(subset=['ts_code'], inplace=True)  # 剔除na值
    df.drop(columns=['start_end'], inplace=True)
    df = df.loc['2010':, ]  # 保留2010年以后的数据
    stock_df_list.append(df)  # 将数据存入列表
    
#     # =将股票数据保存至datas/stock文件夹下
#     stock_code = ''.join(code.split('.')[::-1])
#     df.to_csv(f'../datas/stock/{stock_code}.csv', encoding='gbk')  
    
stock_df = pd.concat(stock_df_list)
stock_df

,ts_code,open,high,low,close,pre_close,change,pct_chg,vol,amount,name,change_reason
2010-01-04,000001.SZ,24.52,24.58,23.68,23.71,24.37,-0.66,-2.7100,241922.76,5.802495e+05,深发展A,完成股改
2010-01-05,000001.SZ,23.75,23.90,22.75,23.30,23.71,-0.41,-1.7300,556499.82,1.293477e+06,深发展A,完成股改
2010-01-06,000001.SZ,23.25,23.25,22.72,22.90,23.30,-0.40,-1.7200,412143.13,9.444537e+05,深发展A,完成股改
2010-01-07,000001.SZ,22.90,23.05,22.40,22.65,22.90,-0.25,-1.0900,355336.85,8.041663e+05,深发展A,完成股改
2010-01-08,000001.SZ,22.50,22.75,22.35,22.60,22.65,-0.05,-0.2200,288543.06,6.506674e+05,深发展A,完成股改
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-27,000011.SZ,23.08,23.83,21.72,21.74,24.13,-2.39,-9.9047,299759.80,6.711774e+05,深物业A,完成股改
2020-07-28,000011.SZ,22.35,22.78,21.85,22.48,21.74,0.74,3.4039,205364.49,4.586896e+05,深物业A,完成股改
2020-07-29,000011.SZ,22.20,23.35,21.64,23.32,22.48,0.84,3.7367,234401.89,5.345514e+05,深物业A,完成股改
2020-07-30,000011.SZ,23.18,25.15,22.80,24.49,23.32,1.17,5.0172,305087.01,7.240126e+05,深物业A,完成股改


In [84]:
stock_df.to_csv('stock_data.csv', encoding='gbk')  # 保存数据